# Submission Notebook
We will use Autogluon combined with some basic cleaning

In [89]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularDataset, TabularPredictor

from numbers import Number

In [80]:
# change number of rows/columns we will view
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Format Data

Read data and do basic cleaning of NaN values

We will only replace values if there is a meaningful replacement; otherwise we leave it for AutoGluon

In [43]:
# read data and convert target variable to log
# log conversion normalizes data (also is where we measure RMSE)

# training
df_train = pd.read_csv('../data/raw/train.csv')
df_train['SalePrice']= np.log(df_train['SalePrice'])

# test
df_test = pd.read_csv('../data/raw/test.csv')
# there is no SalePrice variable (only Kaggle has)


In [45]:
# Find columns containing NaN values
train_columns_with_nan = df_train.columns[df_train.isna().any()].tolist()
test_columns_with_nan = df_test.columns[df_test.isna().any()].tolist()

# Print the columns with NaN values
print("Training Columns with NaN values:", train_columns_with_nan)
print('                   ')
print('that is ' + str(len(train_columns_with_nan)) + ' columns')
print('                   ')
print("Test Columns with NaN values:", test_columns_with_nan)
print('                  ')
print('that is ' + str(len(test_columns_with_nan)) + ' columns')
print('                   ')

# check that training/test are missing same columns
print('Train/Test have same missing columns: ', train_columns_with_nan == test_columns_with_nan)


Training Columns with NaN values: ['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
                   
that is 19 columns
                   
Test Columns with NaN values: ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']
                  
that is 33 columns
                   
Train/Test have same missing columns:  False


In [74]:
# Several features seem really easily fillable
# Categoricals that seem to be NaN because that feature is not on property
# often are other columns that verify this assumption (such as area = 0)

# easy fixes for training
easy_fix_cols = ['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',  'BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']


# Elements in listA but not in listB
not_in_easyFix = [item for item in test_columns_with_nan if item not in easy_fix_cols]

# Elements in listB but not in listA
not_in_testNan = [item for item in easy_fix_cols if item not in test_columns_with_nan]

print("Elements in listA but not in listB:", not_in_easyFix)
print('  ')
print("Elements in listB but not in listA:", not_in_testNan)

Elements in listA but not in listB: ['MSZoning', 'LotFrontage', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'SaleType']
  
Elements in listB but not in listA: []


In [91]:
for colName in not_in_easyFix:
    print(colName)
    print('missing: ', sum(df_test[colName].isna()))
    print(df_test[colName].value_counts())
    if isinstance(df_test[colName][0], Number):
        print('minimum value: ', min(df_test[colName]))
    print('  ')

MSZoning
missing:  4
RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64
  
LotFrontage
missing:  227
60.0     133
80.0      68
70.0      63
50.0      60
75.0      52
        ... 
117.0      1
31.0       1
119.0      1
25.0       1
140.0      1
Name: LotFrontage, Length: 115, dtype: int64
minimum value:  21.0
  
Utilities
missing:  2
AllPub    1457
Name: Utilities, dtype: int64
  
Exterior1st
missing:  1
VinylSd    510
MetalSd    230
HdBoard    220
Wd Sdng    205
Plywood    113
CemntBd     65
BrkFace     37
WdShing     30
AsbShng     24
Stucco      18
BrkComm      4
AsphShn      1
CBlock       1
Name: Exterior1st, dtype: int64
  
Exterior2nd
missing:  1
VinylSd    510
MetalSd    233
HdBoard    199
Wd Sdng    194
Plywood    128
CmentBd     66
Wd Shng     43
BrkFace     22
Stucco      21
AsbShng     18
Brk Cmn     15
ImStucc      5
CBlock       2
AsphShn      1
Stone        1
Name: Exterior2nd, dtype: int64
  
MasVnrArea
missing:  

In [92]:
# Several features seem really easily fillable
# Categoricals that seem to be NaN because that feature is not on property
# often are other columns that verify this assumption (such as area = 0)

# Replace NaN values in the categorical column with 'noFeature'
replacement_value = 'noFeature'
for col in easy_fix_cols:
    df_train[col] = df_train[col].fillna(replacement_value)
    df_test[col] = df_test[col].fillna(replacement_value)

In [93]:
# missing values in 'MasVnrArea' are because there is no masonry veneer

# Find the indices of NaN entries in the 'MasVnrArea' column (train)
column_name = 'MasVnrArea'
nan_indices = df_train[df_train[column_name].isna()].index

# Set the values in the 'MasVnrArea' column for the specified indices to 0 (train)
df_train.loc[nan_indices, column_name] = 0

# Find the indices of NaN entries in the 'MasVnrArea' column (train)
nan_indices = df_test[df_test[column_name].isna()].index

# Set the values in the 'MasVnrArea' column for the specified indices to 0 (train)
df_test.loc[nan_indices, column_name] = 0

In [94]:
# Assume missing 'LotFrontage' Values have no lot frontage
df_train['LotFrontage'] = df_train['LotFrontage'].fillna(0)
df_test['LotFrontage'] = df_test['LotFrontage'].fillna(0)

## Autogluon



In [95]:
# convert to tabular databases

td_test = TabularDataset(df_test)
td_train = TabularDataset(df_train)

In [96]:
# train predictor
predictor = TabularPredictor(label='SalePrice').fit(td_train)

No path specified. Models will be saved in: "AutogluonModels/ag-20231107_235450/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231107_235450/"
AutoGluon Version:  0.8.2
Python Version:     3.9.16
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 21.6.0: Mon Aug 22 20:17:10 PDT 2022; root:xnu-8020.140.49~2/RELEASE_X86_64
Disk Space Avail:   17.45 GB / 250.69 GB (7.0%)
Train Data Rows:    1460
Train Data Columns: 80
Label Column: SalePrice
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (13.534473028231162, 10.460242108190519, 12.02405, 0.39945)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Fea

[1000]	valid_set's rmse: 0.127134


	-0.1269	 = Validation score   (-root_mean_squared_error)
	1.66s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 0.126613
[2000]	valid_set's rmse: 0.125737
[3000]	valid_set's rmse: 0.12561
[4000]	valid_set's rmse: 0.125581
[5000]	valid_set's rmse: 0.125575
[6000]	valid_set's rmse: 0.125573
[7000]	valid_set's rmse: 0.125573
[8000]	valid_set's rmse: 0.125573


	-0.1256	 = Validation score   (-root_mean_squared_error)
	19.39s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.1348	 = Validation score   (-root_mean_squared_error)
	3.01s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost ...
	-0.1264	 = Validation score   (-root_mean_squared_error)
	202.05s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.1344	 = Validation score   (-root_mean_squared_error)
	3.06s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.1523	 = Validation score   (-root_mean_squared_error)
	5.1s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	-0.1193	 = Validation score   (-root_mean_squared_error)
	5.76s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-0.1472	 = Validation score   (-root_mean_squared_error)
	17.52s	 = Training   runtime
	0.05s	 = Validation ru

[1000]	valid_set's rmse: 0.138771
[2000]	valid_set's rmse: 0.138765


	-0.1388	 = Validation score   (-root_mean_squared_error)
	15.06s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.1157	 = Validation score   (-root_mean_squared_error)
	0.49s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 277.78s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231107_235450/")


In [97]:
# make prediction

# must exp to undo earlier log
myPred = np.exp(predictor.predict(td_test))



In [98]:
myPred

0       123037.609375
1       159437.968750
2       188119.484375
3       192725.578125
4       181773.296875
            ...      
1454     79196.992188
1455     79601.414062
1456    166950.015625
1457    108950.187500
1458    215446.937500
Name: SalePrice, Length: 1459, dtype: float32

In [100]:
# export to csv

# re-indexing to deal with submission requirements
myPred.index = range(1461, 1461 + len(myPred))

path = 'submission1.csv'

myPred.reset_index().to_csv(path, header = ['Id', 'SalePrice'], index = False)